In [ ]:
# ✅ Install required packages
!pip install mediapipe deepface librosa matplotlib fpdf pandas opencv-python --quiet
!apt-get install -y ffmpeg --quiet


In [ ]:
# ✅ Set video path (upload your .mov file to the Files tab first)
video_path = "movie_test.mov"
audio_path = "/mnt/data/audio.wav"

import os, cv2, numpy as np, librosa, matplotlib.pyplot as plt
import mediapipe as mp, pandas as pd, shutil
from datetime import datetime
from deepface import DeepFace
from fpdf import FPDF
from scipy.signal import find_peaks
import subprocess


In [ ]:
# 🎙️ Extract audio
os.makedirs("/mnt/data", exist_ok=True)
subprocess.call(['ffmpeg', '-i', video_path, '-ab', '160k', '-ac', '2', '-ar', '44100', '-vn', audio_path])


In [ ]:
# 📊 Audio analysis
y, sr = librosa.load(audio_path, sr=None)
rms = librosa.feature.rms(y=y)[0]
rms_std = np.std(rms)
pitch, mag = librosa.piptrack(y=y, sr=sr)
pitch_values = pitch[mag > np.median(mag)]
pitch_std = np.std(pitch_values) if len(pitch_values) > 0 else 0
intervals = librosa.effects.split(y, top_db=30)
gaps = [(intervals[i][0] - intervals[i-1][1]) / sr for i in range(1, len(intervals))]
avg_latency = np.mean(gaps) if gaps else 0
cognitive_load = np.var(rms)


In [ ]:
# 👁️ Blink detection using EAR
cap = cv2.VideoCapture(video_path)
frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
mp_face_mesh = mp.solutions.face_mesh
ear_values = []

def calc_ear(landmarks, indices):
    try:
        p = lambda i: np.array([landmarks[i].x, landmarks[i].y])
        vertical = np.linalg.norm(p(indices[1]) - p(indices[5])) + np.linalg.norm(p(indices[2]) - p(indices[4]))
        horizontal = 2 * np.linalg.norm(p(indices[0]) - p(indices[3]))
        return vertical / horizontal if horizontal != 0 else 0
    except: return 0

with mp_face_mesh.FaceMesh(static_image_mode=True) as mesh:
    for i in range(0, frame_count, max(1, frame_count // 30)):
        cap.set(cv2.CAP_PROP_POS_FRAMES, i)
        ret, f = cap.read()
        if not ret: continue
        rgb = cv2.cvtColor(f, cv2.COLOR_BGR2RGB)
        result = mesh.process(rgb)
        if result.multi_face_landmarks:
            lmk = result.multi_face_landmarks[0].landmark
            ear = calc_ear(lmk, [33, 160, 158, 133, 153, 144])
            ear_values.append(ear)

blinks, _ = find_peaks(np.array(ear_values), prominence=0.01)
blink_rate = len(blinks)


In [ ]:
# 😊 Emotion detection
cap.set(cv2.CAP_PROP_POS_FRAMES, frame_count // 2)
ret, frame = cap.read()
emotion_result = DeepFace.analyze(frame, actions=['emotion'], enforce_detection=False)
dominant_emotion = emotion_result[0]['dominant_emotion']
emotion_probs = emotion_result[0]['emotion']
emotion_chart = list(emotion_probs.items())


In [ ]:
# 📈 Save graphs
def save_graph(func, path):
    func()
    plt.tight_layout()
    plt.savefig(path)
    plt.close()

save_graph(lambda: plt.barh([x[0] for x in emotion_chart], [x[1] for x in emotion_chart]), "emotion_chart.png")
save_graph(lambda: plt.plot(ear_values), "blink_graph.png")
save_graph(lambda: plt.plot(rms[:300]), "rms_chart.png")
save_graph(lambda: plt.hist(pitch_values, bins=50), "pitch_histogram.png")

signal_vals = [pitch_std/800, rms_std/0.2, avg_latency/2, blink_rate/50, cognitive_load/0.1]
save_graph(lambda: plt.barh(range(5), signal_vals), "truthmatch_score.png")

truth_score = round(1 - np.mean(signal_vals), 3)
truth_score = min(max(truth_score, 0.0), 1.0)


In [ ]:
# 🧾 PDF Report
pdf = FPDF()
pdf.set_auto_page_break(auto=True, margin=10)
pdf.add_page()
pdf.set_font("Arial", 'B', 16)
pdf.cell(0, 10, "TruthMark-Aurion Forensic Report", ln=True)
pdf.set_font("Arial", size=12)
pdf.cell(0, 10, "Scan Date: " + datetime.now().strftime("%Y-%m-%d %H:%M:%S"), ln=True)
pdf.ln(10)

pdf.multi_cell(0, 10, f"""
• Dominant Emotion: {dominant_emotion}
• Blink Rate: {blink_rate}
• Voice Tremor Std Dev: {pitch_std:.2f} Hz
• Emotion RMS Std Dev: {rms_std:.4f}
• Speech Latency Avg: {avg_latency:.2f} sec
• Cognitive Load Proxy: {cognitive_load:.5f}
• TruthMatch Score: {truth_score:.3f}
""")

for graph in ["emotion_chart.png", "blink_graph.png", "rms_chart.png", "pitch_histogram.png", "truthmatch_score.png"]:
    if os.path.exists(graph):
        pdf.add_page()
        pdf.image(graph, x=10, w=180)

pdf.output("TruthMark_Aurion_Master_Report.pdf")
print("✅ PDF saved as TruthMark_Aurion_Master_Report.pdf")
